In [1]:
from inference import SparseModel

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SparseModel()

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
QDRANT_URL='http://localhost:6333'
QDRANT_API_KEY=''
COLLECTION_NAME="quora"

In [92]:
from qdrant_client import QdrantClient, models
import math

In [101]:
def rescore_vector(vector: dict) -> dict:
    new_vector = {}

    sorted_vector = sorted(vector.items(), key=lambda x: x[1], reverse=True)

    for num, (idx, _value) in enumerate(sorted_vector):
        new_vector[idx] = math.log(1./(num + 1) + 1.) # * value
    return new_vector


In [102]:
def conver_sparse_vector(sparse_vector: dict) -> models.SparseVector:
    indices = []
    values = []

    for (idx, value) in sparse_vector.items():
        indices.append(int(idx))
        values.append(value)

    return models.SparseVector(
        indices=indices,
        values=values
    )

In [103]:
client = QdrantClient(QDRANT_URL, api_key=QDRANT_API_KEY)

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:122: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


In [104]:
query = "Do you think of mathematics as art or science?"
sparse_vector = rescore_vector(next(model.encode([query])))
sparse_vector_qdrant = conver_sparse_vector(sparse_vector)

limit = 20
result = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=models.NamedSparseVector(
        name="attention",
        vector=sparse_vector_qdrant
    ),
    with_payload=True,
    with_vectors=True,
    limit=limit
)

In [105]:
for idx, hit in enumerate(result):
    print(idx, hit)

0 id=435817 version=19847 score=3.8775964 payload={'text': 'What are the similarities between philosophy of science and philosophy of mathematics?'} vector={'attention': SparseVector(indices=[1029, 1996, 1997, 1998, 2024, 2054, 2090, 2671, 4695, 5597, 12319], values=[0.0002131269, 0.12949592, 0.30499616, 0.17941843, 0.1838402, 0.11063437, 0.31784216, 2.0730793, 2.0757017, 4.6819096, 1.6534677])} shard_key=None
1 id=142399 version=6473 score=3.841679 payload={'text': 'Is mathematics always necessary in doing good science?'} vector={'attention': SparseVector(indices=[1029, 1999, 2003, 2204, 2467, 2671, 2725, 4072, 5597], values=[0.0002520756, 0.16922376, 0.149325, 0.9866759, 0.8835106, 2.0730793, 0.5679861, 1.4954723, 4.6819096])} shard_key=None
2 id=415418 version=18914 score=3.8416653 payload={'text': 'What are career options for a science student with mathematics except engineering?'} vector={'attention': SparseVector(indices=[1029, 1037, 2005, 2007, 2024, 2054, 2476, 2671, 3076, 3272

In [110]:
for (idx, value) in zip(sparse_vector_qdrant.indices, sparse_vector_qdrant.values):
    print(f"{model.invert_vocab[idx]}: {value}")

mathematics: 0.6931471805599453
art: 0.4054651081081644
science: 0.28768207245178085
think: 0.22314355131420976
as: 0.1823215567939546
?: 0.15415067982725836
or: 0.13353139262452257
of: 0.11778303565638346
you: 0.10536051565782635
do: 0.09531017980432493


In [108]:
result_sparse_vector = result[19].vector['attention']

for (idx, value) in zip(result_sparse_vector.indices, result_sparse_vector.values):
    if idx in sparse_vector:
        print(f"{model.invert_vocab[idx]}: {value} x {sparse_vector[idx]} = {value * sparse_vector[idx]}")
    else:
        print(f"{model.invert_vocab[idx]}: {value}")


?: 0.00030851562 x 0.15415067982725836 = 4.75578925603281e-05
of: 0.25787058 x 0.11778303565638346 = 0.030372779718872284
you: 0.31449428 x 0.10536051565782635 = 0.033135279512236826
can: 0.1911425
how: 0.14706846
##ia: 1.1969953
mathematics: 4.6819096 x 0.6931471805599453 = 3.245252438876541
ph: 1.8742455
overcome: 0.7665973
##ob: 3.348426


In [82]:
query = "Is math an art or a science?"
sv = next(model.encode([query]))
sv = conver_sparse_vector(sv)

for (idx, value) in zip(sv.indices, sv.values):
    print(f"{model.invert_vocab[idx]}: {value}")


is: 0.027513518929481506
math: 0.2162715643644333
an: 0.02588239312171936
art: 0.1304435133934021
or: 0.02149391360580921
a: 0.021214833483099937
science: 0.11642558127641678
?: 0.028545841574668884


In [40]:
import math

In [52]:
math.log(1/20 + 1)

0.04879016416943205

In [112]:
a = [1,3,2]

sorted(a, reverse=True)

a

[1, 3, 2]